In [1]:
import sys
import time
import os
import tensorflow as tf
import numpy as np

from collections import namedtuple
from tensorflow.python import debug as tf_debug
from tensorflow.core.example import example_pb2
from importlib import reload

import json, os, re, shutil, sys, time
import collections, itertools


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import glob
import random
import struct
import csv
import batch; reload(batch)
import data; reload(data)
import model; reload(model)

from batch import Example,Batch
from data import Vocab
from model import SummarizationModel
from decode import BeamSearchDecoder
from batcher import Batcher
import util
import training_util as tutil

In [3]:
def setup_params():
    hps_dict = {
        'mode' : 'train',
        'single_pass' : False,
        'log_root' : '/home/ubuntu/W266/final_0/W266_Final/model_4/saved/train',
        'exp_name' : '',
        'hidden_dim' : 256,
        'emb_dim' : 128,
        'batch_size' : 100,
        'max_enc_steps' : 400,
        'max_dec_steps' : 100,
        'beam_size' : 4,
        'min_dec_steps' : 35,
        'vocab_size' : 50000,
        'lr' : 0.15,
        'adagrad_init_acc' : 0.1,
        'rand_unif_init_mag' : 0.02,
        'trunc_norm_init_std' : 1e-4,
        'max_grad_norm' : 2.0,
        'pointer_gen' : True,
        'coverage' : True,
        'cov_loss_wt' : 1.0,
        'vocab_path' : '/home/ubuntu/W266/final_0/W266_Final/data/final_processed/vocab',
        'data_path' : '/home/ubuntu/W266/final_0/W266_Final/data/final_chunked/train_*'
    }
    
    hps = namedtuple("HParams", hps_dict.keys())(**hps_dict)
    return hps

In [4]:
hps = setup_params()

In [5]:
def train(hps,epochs,restore=False):
    
    lm,vocab,batches,train_dir = tutil.training_init(hps)
    
    with lm.graph.as_default():
        initializer = tf.global_variables_initializer()
        
    with tf.Session(graph=lm.graph) as session:
        
        session.run(initializer)
        saver = tf.train.Saver(max_to_keep=3) # keep 3 checkpoints at a time
        sv = tf.train.Supervisor(logdir=train_dir,
                                 is_chief=True,
                                 saver=saver,
                                 summary_op=None,
                                 save_summaries_secs=30, 
                                 save_model_secs=30, 
                                 global_step=lm.global_step)    

        summary_writer = sv.summary_writer
        
        if restore:
            ckpt_path = util.load_ckpt(saver, session,hps,hps.log_root)

        start_time = time.time()
        train_step = None
        curr_best = None
        best_loss = None
        avg_loss = 0
        for i in range(epochs):
            print(f"[EPOCH {i+1}] Starting training..")
            random.shuffle(batches)
            avg_loss,curr_best,train_step = tutil.run_epoch(lm,
                                                            session,
                                                            batches,
                                                            summary_writer,
                                                            train_dir,
                                                            train_step,
                                                            saver,
                                                            hps,
                                                            best_loss,
                                                            avg_loss)
            
            if(best_loss is None or curr_best < best_loss):
                best_loss = curr_best
            print(f"[EPOCH {i+1}] Complete. Avg Loss: {avg_loss}; Best Loss: {best_loss}")
        sv.stop()
        time_total = tutil.pretty_timedelta(since=start_time)
        print(f"[END] Training complete: Best Loss: {best_loss}; Total time: {time_total}")


In [6]:
train(hps,30)







max_size of vocab was specified as 50000; we now have 50000 words. Stopping reading.
Finished constructing vocabulary of 50000 total words. Last word added: 16:03
INFO:tensorflow:Fetching data..
INFO:tensorflow:Creating batches..
INFO:tensorflow:[TOTAL Batches]  : 1872
INFO:tensorflow:[TOTAL Examples] : 187193
INFO:tensorflow:Creating batches..COMPLETE
INFO:tensorflow:Building core graph...
INFO:tensorflow:Adding attention_decoder timestep 0 of 100
INFO:tensorflow:Adding attention_decoder timestep 1 of 100
INFO:tensorflow:Adding attention_decoder timestep 2 of 100
INFO:tensorflow:Adding attention_decoder timestep 3 of 100
INFO:tensorflow:Adding attention_decoder timestep 4 of 100
INFO:tensorflow:Adding attention_decoder timestep 5 of 100
INFO:tensorflow:Adding attention_decoder timestep 6 of 100
INFO:tensorflow:Adding attention_decoder timestep 7 of 100
INFO:tensorflow:Adding attention_decoder timestep 8 of 100
INFO:tensorflow:Adding attention_decoder timestep 9 of 100
INFO:tenso

    [batch 45]: seen 4500 examples at 18.6 eps, loss = 7.420
    [batch 48]: seen 4800 examples at 18.9 eps, loss = 7.417
    [batch 51]: seen 5100 examples at 19.0 eps, loss = 7.413
    [batch 53]: seen 5300 examples at 19.0 eps, loss = 7.411
    [batch 56]: seen 5600 examples at 19.3 eps, loss = 7.410
    [batch 60]: seen 6000 examples at 19.8 eps, loss = 7.409
    [batch 63]: seen 6300 examples at 19.9 eps, loss = 7.407
    [batch 66]: seen 6600 examples at 19.9 eps, loss = 7.404
    [batch 69]: seen 6900 examples at 20.1 eps, loss = 7.404
    [batch 74]: seen 7400 examples at 20.7 eps, loss = 7.401
    [batch 77]: seen 7700 examples at 20.7 eps, loss = 7.398
    [batch 80]: seen 8000 examples at 20.7 eps, loss = 7.394
    [batch 83]: seen 8300 examples at 20.8 eps, loss = 7.392
    [batch 86]: seen 8600 examples at 21.0 eps, loss = 7.391
    [batch 89]: seen 8900 examples at 21.0 eps, loss = 7.389
    [batch 92]: seen 9200 examples at 21.1 eps, loss = 7.387
    [batch 95]: seen 950

    [batch 432]: seen 43200 examples at 21.8 eps, loss = 6.969
    [batch 435]: seen 43500 examples at 21.8 eps, loss = 6.966
    [batch 437]: seen 43700 examples at 21.8 eps, loss = 6.962
    [batch 440]: seen 44000 examples at 21.8 eps, loss = 6.959
    [batch 443]: seen 44300 examples at 21.8 eps, loss = 6.957
    [batch 446]: seen 44600 examples at 21.7 eps, loss = 6.953
    [batch 449]: seen 44900 examples at 21.7 eps, loss = 6.949
    [batch 452]: seen 45200 examples at 21.7 eps, loss = 6.946
    [batch 454]: seen 45400 examples at 21.7 eps, loss = 6.944
    [batch 457]: seen 45700 examples at 21.7 eps, loss = 6.942
    [batch 460]: seen 46000 examples at 21.7 eps, loss = 6.939
    [batch 463]: seen 46300 examples at 21.7 eps, loss = 6.933
    [batch 466]: seen 46600 examples at 21.7 eps, loss = 6.929
    [batch 469]: seen 46900 examples at 21.7 eps, loss = 6.927
    [batch 471]: seen 47100 examples at 21.7 eps, loss = 6.924
    [batch 474]: seen 47400 examples at 21.7 eps, loss 

INFO:tensorflow:Restoring parameters from /home/ubuntu/W266/final_0/W266_Final/model_4/saved/train/model-879
    [batch 881]: seen 88100 examples at 24.3 eps, loss = 6.743
    [batch 883]: seen 88300 examples at 24.3 eps, loss = 6.741
    [batch 886]: seen 88600 examples at 24.3 eps, loss = 6.740
    [batch 893]: seen 89300 examples at 24.4 eps, loss = 6.742
    [batch 896]: seen 89600 examples at 24.4 eps, loss = 6.739
    [batch 899]: seen 89900 examples at 24.4 eps, loss = 6.738
    [batch 903]: seen 90300 examples at 24.4 eps, loss = 6.736
    [batch 906]: seen 90600 examples at 24.4 eps, loss = 6.734
    [batch 913]: seen 91300 examples at 24.5 eps, loss = 6.734
    [batch 917]: seen 91700 examples at 24.6 eps, loss = 6.734
    [batch 920]: seen 92000 examples at 24.5 eps, loss = 6.731
    [batch 925]: seen 92500 examples at 24.6 eps, loss = 6.729
    [batch 932]: seen 93200 examples at 24.7 eps, loss = 6.732
    [batch 939]: seen 93900 examples at 24.8 eps, loss = 6.734
    [batc

    [batch 1381]: seen 138100 examples at 26.8 eps, loss = 6.588
    [batch 1384]: seen 138400 examples at 26.8 eps, loss = 6.587
    [batch 1391]: seen 139100 examples at 26.9 eps, loss = 6.590
    [batch 1398]: seen 139800 examples at 27.0 eps, loss = 6.588
    [batch 1403]: seen 140300 examples at 27.0 eps, loss = 6.586
    [batch 1406]: seen 140600 examples at 27.0 eps, loss = 6.586
    [batch 1411]: seen 141100 examples at 27.1 eps, loss = 6.586
    [batch 1418]: seen 141800 examples at 27.2 eps, loss = 6.586
    [batch 1423]: seen 142300 examples at 27.2 eps, loss = 6.584
    [batch 1430]: seen 143000 examples at 27.3 eps, loss = 6.585
    [batch 1437]: seen 143700 examples at 27.4 eps, loss = 6.585
    [batch 1440]: seen 144000 examples at 27.4 eps, loss = 6.583
    [batch 1443]: seen 144300 examples at 27.3 eps, loss = 6.582
    [batch 1446]: seen 144600 examples at 27.3 eps, loss = 6.581
    [batch 1449]: seen 144900 examples at 27.3 eps, loss = 6.580
    [batch 1452]: seen 14

...

    [batch 1448]: seen 144800 examples at 48.0 eps, loss = 4.312
    [batch 1455]: seen 145500 examples at 48.1 eps, loss = 4.315
    [batch 1460]: seen 146000 examples at 48.1 eps, loss = 4.311
    [batch 1465]: seen 146500 examples at 48.0 eps, loss = 4.309
    [batch 1472]: seen 147200 examples at 48.1 eps, loss = 4.311
    [batch 1479]: seen 147900 examples at 48.1 eps, loss = 4.315
    [batch 1486]: seen 148600 examples at 48.2 eps, loss = 4.310
    [batch 1491]: seen 149100 examples at 48.2 eps, loss = 4.307
    [batch 1498]: seen 149800 examples at 48.2 eps, loss = 4.313
    [batch 1505]: seen 150500 examples at 48.3 eps, loss = 4.309
    [batch 1510]: seen 151000 examples at 48.3 eps, loss = 4.309
    [batch 1517]: seen 151700 examples at 48.4 eps, loss = 4.308
    [batch 1524]: seen 152400 examples at 48.4 eps, loss = 4.308
    [batch 1529]: seen 152900 examples at 48.4 eps, loss = 4.305
    [batch 1536]: seen 153600 examples at 48.5 eps, loss = 4.309
    [batch 1543]: seen 15

    [batch 279]: seen 27900 examples at 52.1 eps, loss = 4.204
    [batch 285]: seen 28500 examples at 52.0 eps, loss = 4.203
    [batch 289]: seen 28900 examples at 51.6 eps, loss = 4.199
    [batch 294]: seen 29400 examples at 51.5 eps, loss = 4.201
    [batch 300]: seen 30000 examples at 51.5 eps, loss = 4.199
    [batch 307]: seen 30700 examples at 51.8 eps, loss = 4.204
    [batch 314]: seen 31400 examples at 52.1 eps, loss = 4.199
    [batch 321]: seen 32100 examples at 52.3 eps, loss = 4.205
    [batch 328]: seen 32800 examples at 52.6 eps, loss = 4.199
    [batch 331]: seen 33100 examples at 52.2 eps, loss = 4.197
    [batch 338]: seen 33800 examples at 52.2 eps, loss = 4.196
    [batch 345]: seen 34500 examples at 52.4 eps, loss = 4.197
    [batch 348]: seen 34800 examples at 52.0 eps, loss = 4.196
    [batch 355]: seen 35500 examples at 52.3 eps, loss = 4.203
    [batch 362]: seen 36200 examples at 52.5 eps, loss = 4.198
    [batch 365]: seen 36500 examples at 52.1 eps, loss 

    [batch 1056]: seen 105600 examples at 56.1 eps, loss = 4.144
    [batch 1061]: seen 106100 examples at 56.0 eps, loss = 4.143
    [batch 1064]: seen 106400 examples at 55.8 eps, loss = 4.141
    [batch 1071]: seen 107100 examples at 55.8 eps, loss = 4.143
    [batch 1075]: seen 107500 examples at 55.7 eps, loss = 4.141
    [batch 1078]: seen 107800 examples at 55.5 eps, loss = 4.142
    [batch 1084]: seen 108400 examples at 55.5 eps, loss = 4.138
    [batch 1091]: seen 109100 examples at 55.6 eps, loss = 4.141
    [batch 1096]: seen 109600 examples at 55.5 eps, loss = 4.137
    [batch 1099]: seen 109900 examples at 55.4 eps, loss = 4.137
    [batch 1106]: seen 110600 examples at 55.4 eps, loss = 4.152
    [batch 1113]: seen 111300 examples at 55.5 eps, loss = 4.155
    [batch 1120]: seen 112000 examples at 55.6 eps, loss = 4.154
    [batch 1127]: seen 112700 examples at 55.6 eps, loss = 4.151
    [batch 1134]: seen 113400 examples at 55.7 eps, loss = 4.148
    [batch 1141]: seen 11

    [batch 1833]: seen 183300 examples at 56.4 eps, loss = 4.076
    [batch 1836]: seen 183600 examples at 56.3 eps, loss = 4.076
    [batch 1841]: seen 184100 examples at 56.2 eps, loss = 4.074
    [batch 1848]: seen 184800 examples at 56.2 eps, loss = 4.076
    [batch 1855]: seen 185500 examples at 56.3 eps, loss = 4.076
    [batch 1858]: seen 185800 examples at 56.2 eps, loss = 4.072
    [batch 1861]: seen 186100 examples at 56.0 eps, loss = 4.068
    [batch 1868]: seen 186800 examples at 56.1 eps, loss = 4.071
    [END] Training complete: Total examples : 187100; Total time: 0:55:36
[EPOCH 5] Complete. Avg Loss: 4.073161114532455; Best Loss: 4.067813832117995
[EPOCH 6] Starting training..
    [batch 7]: seen 700 examples at 68.8 eps, loss = 4.075


KeyboardInterrupt: 


# Training session 2

In [5]:
def train_continue(hps,epochs,train_step,curr_best,best_loss,avg_loss,restore,epoch_start):
    
    lm,vocab,batches,train_dir = tutil.training_init(hps)
    
    with lm.graph.as_default():
        initializer = tf.global_variables_initializer()
        
    with tf.Session(graph=lm.graph) as session:
        
        session.run(initializer)
        saver = tf.train.Saver(max_to_keep=3) # keep 3 checkpoints at a time
        sv = tf.train.Supervisor(logdir=train_dir,
                                 is_chief=True,
                                 saver=saver,
                                 summary_op=None,
                                 save_summaries_secs=30, 
                                 save_model_secs=30, 
                                 global_step=lm.global_step)    

        summary_writer = sv.summary_writer
        
        if restore:
            ckpt_path = util.load_ckpt(saver, session,hps,hps.log_root)

        start_time = time.time()
        for i in range(epoch_start,epoch_start+epochs):
            print(f"[EPOCH {i+1}] Starting training..")
            random.shuffle(batches)
            avg_loss,curr_best,train_step = tutil.run_epoch(lm,
                                                            session,
                                                            batches,
                                                            summary_writer,
                                                            train_dir,
                                                            train_step,
                                                            saver,
                                                            hps,
                                                            best_loss,
                                                            avg_loss)
            
            if(best_loss is None or curr_best < best_loss):
                best_loss = curr_best
            print(f"[EPOCH {i+1}] Complete. Avg Loss: {avg_loss}; Best Loss: {best_loss}")
        sv.stop()
        time_total = tutil.pretty_timedelta(since=start_time)
        print(f"[END] Training complete: Best Loss: {best_loss}; Total time: {time_total}")


In [6]:
avg_loss = 4.07316
best_loss = 4.06781
curr_best = best_loss
train_step = 9260
epochs = 5
restore = True
epoch_start = 5

train_continue(hps,epochs,train_step,curr_best,best_loss,avg_loss,restore,epoch_start)







max_size of vocab was specified as 50000; we now have 50000 words. Stopping reading.
Finished constructing vocabulary of 50000 total words. Last word added: 16:03
INFO:tensorflow:Fetching data..
INFO:tensorflow:Creating batches..
INFO:tensorflow:[TOTAL Batches]  : 1872
INFO:tensorflow:[TOTAL Examples] : 187193
INFO:tensorflow:Creating batches..COMPLETE
INFO:tensorflow:Building core graph...
INFO:tensorflow:Adding attention_decoder timestep 0 of 100
INFO:tensorflow:Adding attention_decoder timestep 1 of 100
INFO:tensorflow:Adding attention_decoder timestep 2 of 100
INFO:tensorflow:Adding attention_decoder timestep 3 of 100
INFO:tensorflow:Adding attention_decoder timestep 4 of 100
INFO:tensorflow:Adding attention_decoder timestep 5 of 100
INFO:tensorflow:Adding attention_decoder timestep 6 of 100
INFO:tensorflow:Adding attention_decoder timestep 7 of 100
INFO:tensorflow:Adding attention_decoder timestep 8 of 100
INFO:tensorflow:Adding attention_decoder timestep 9 of 100
INFO:tenso

    [batch 65]: seen 6500 examples at 41.0 eps, loss = 4.059
    [batch 68]: seen 6800 examples at 39.3 eps, loss = 4.055
    [batch 73]: seen 7300 examples at 39.7 eps, loss = 4.054
    [batch 75]: seen 7500 examples at 38.6 eps, loss = 4.053
    [batch 80]: seen 8000 examples at 39.0 eps, loss = 4.056
    [batch 87]: seen 8700 examples at 40.4 eps, loss = 4.055
    [batch 94]: seen 9400 examples at 41.7 eps, loss = 4.057
    [batch 101]: seen 10100 examples at 42.9 eps, loss = 4.058
    [batch 108]: seen 10800 examples at 44.0 eps, loss = 4.060
    [batch 115]: seen 11500 examples at 45.0 eps, loss = 4.065
    [batch 122]: seen 12200 examples at 45.9 eps, loss = 4.061
    [batch 129]: seen 12900 examples at 46.8 eps, loss = 4.062
    [batch 136]: seen 13600 examples at 47.6 eps, loss = 4.064
    [batch 143]: seen 14300 examples at 48.3 eps, loss = 4.068
    [batch 150]: seen 15000 examples at 49.0 eps, loss = 4.069
    [batch 157]: seen 15700 examples at 49.7 eps, loss = 4.066
    [b

INFO:tensorflow:Restoring parameters from /home/ubuntu/W266/final_0/W266_Final/model_4/saved/train/model-9798
    [batch 630]: seen 63000 examples at 54.4 eps, loss = 4.014
    [batch 637]: seen 63700 examples at 54.6 eps, loss = 4.014
    [batch 644]: seen 64400 examples at 54.7 eps, loss = 4.015
    [batch 651]: seen 65100 examples at 54.8 eps, loss = 4.014
    [batch 656]: seen 65600 examples at 54.8 eps, loss = 4.011
    [batch 660]: seen 66000 examples at 54.5 eps, loss = 4.010
    [batch 667]: seen 66700 examples at 54.6 eps, loss = 4.011
    [batch 672]: seen 67200 examples at 54.6 eps, loss = 4.010
    [batch 679]: seen 67900 examples at 54.7 eps, loss = 4.013
    [batch 686]: seen 68600 examples at 54.8 eps, loss = 4.010
    [batch 691]: seen 69100 examples at 54.8 eps, loss = 4.011
    [batch 696]: seen 69600 examples at 54.7 eps, loss = 4.009
    [batch 700]: seen 70000 examples at 54.5 eps, loss = 4.008
    [batch 705]: seen 70500 examples at 54.4 eps, loss = 4.008
    [bat

    [batch 1429]: seen 142900 examples at 57.3 eps, loss = 3.975
    [batch 1436]: seen 143600 examples at 57.3 eps, loss = 3.984
    [batch 1443]: seen 144300 examples at 57.4 eps, loss = 3.978
    [batch 1449]: seen 144900 examples at 57.3 eps, loss = 3.975
    [batch 1452]: seen 145200 examples at 57.2 eps, loss = 3.975
    [batch 1455]: seen 145500 examples at 57.1 eps, loss = 3.972
    [batch 1462]: seen 146200 examples at 57.1 eps, loss = 3.972
    [batch 1465]: seen 146500 examples at 57.0 eps, loss = 3.970
    [batch 1472]: seen 147200 examples at 57.1 eps, loss = 3.977
    [batch 1479]: seen 147900 examples at 57.1 eps, loss = 3.974
    [batch 1486]: seen 148600 examples at 57.2 eps, loss = 3.975
    [batch 1494]: seen 149400 examples at 57.2 eps, loss = 3.970
    [batch 1497]: seen 149700 examples at 57.1 eps, loss = 3.969
    [batch 1504]: seen 150400 examples at 57.1 eps, loss = 3.974
    [batch 1511]: seen 151100 examples at 57.2 eps, loss = 3.977
    [batch 1519]: seen 15

INFO:tensorflow:Restoring parameters from /home/ubuntu/W266/final_0/W266_Final/model_4/saved/train/model-10665
    [batch 1591]: seen 159100 examples at 57.5 eps, loss = 3.989
    [batch 1598]: seen 159800 examples at 57.5 eps, loss = 3.989
    [batch 1606]: seen 160600 examples at 57.6 eps, loss = 3.982
    [batch 1614]: seen 161400 examples at 57.6 eps, loss = 3.980
    [batch 1622]: seen 162200 examples at 57.7 eps, loss = 3.981
    [batch 1629]: seen 162900 examples at 57.7 eps, loss = 3.985
    [batch 1637]: seen 163700 examples at 57.8 eps, loss = 3.980
    [batch 1645]: seen 164500 examples at 57.8 eps, loss = 3.979
    [batch 1653]: seen 165300 examples at 57.9 eps, loss = 3.979
    [batch 1661]: seen 166100 examples at 57.9 eps, loss = 3.982
    [batch 1669]: seen 166900 examples at 58.0 eps, loss = 3.982
    [batch 1677]: seen 167700 examples at 58.0 eps, loss = 3.983
    [batch 1685]: seen 168500 examples at 58.1 eps, loss = 3.985
    [batch 1693]: seen 169300 examples at 58

...

INFO:tensorflow:Restoring parameters from /home/ubuntu/W266/final_0/W266_Final/model_4/saved/train/model-13574
    [batch 1118]: seen 111800 examples at 66.2 eps, loss = 3.816
    [batch 1123]: seen 112300 examples at 66.1 eps, loss = 3.814
    [batch 1126]: seen 112600 examples at 65.9 eps, loss = 3.812
    [batch 1128]: seen 112800 examples at 65.6 eps, loss = 3.811
    [batch 1135]: seen 113500 examples at 65.6 eps, loss = 3.813
    [batch 1138]: seen 113800 examples at 65.4 eps, loss = 3.809
    [batch 1141]: seen 114100 examples at 65.1 eps, loss = 3.806
    [batch 1144]: seen 114400 examples at 64.9 eps, loss = 3.805
    [batch 1152]: seen 115200 examples at 64.9 eps, loss = 3.810
    [batch 1159]: seen 115900 examples at 65.0 eps, loss = 3.805
    [batch 1164]: seen 116400 examples at 64.9 eps, loss = 3.806
    [batch 1172]: seen 117200 examples at 64.9 eps, loss = 3.812
    [batch 1179]: seen 117900 examples at 64.9 eps, loss = 3.813
    [batch 1187]: seen 118700 examples at 65

INFO:tensorflow:Restoring parameters from /home/ubuntu/W266/final_0/W266_Final/model_4/saved/train/model-13619
    [batch 1308]: seen 130800 examples at 65.0 eps, loss = 3.830
    [batch 1316]: seen 131600 examples at 65.1 eps, loss = 3.839
    [batch 1324]: seen 132400 examples at 65.1 eps, loss = 3.848
    [batch 1332]: seen 133200 examples at 65.1 eps, loss = 3.850
    [batch 1340]: seen 134000 examples at 65.1 eps, loss = 3.846
    [batch 1348]: seen 134800 examples at 65.2 eps, loss = 3.846
    [batch 1356]: seen 135600 examples at 65.2 eps, loss = 3.844
    [batch 1364]: seen 136400 examples at 65.2 eps, loss = 3.846
    [batch 1372]: seen 137200 examples at 65.3 eps, loss = 3.844
    [batch 1380]: seen 138000 examples at 65.3 eps, loss = 3.846
    [batch 1387]: seen 138700 examples at 65.3 eps, loss = 3.841
    [batch 1395]: seen 139500 examples at 65.3 eps, loss = 3.836
    [batch 1402]: seen 140200 examples at 65.3 eps, loss = 3.836
    [batch 1409]: seen 140900 examples at 65

    [batch 368]: seen 36800 examples at 70.0 eps, loss = 3.810
    [batch 376]: seen 37600 examples at 70.0 eps, loss = 3.813
    [batch 384]: seen 38400 examples at 70.0 eps, loss = 3.815
    [batch 391]: seen 39100 examples at 70.0 eps, loss = 3.811
    [batch 399]: seen 39900 examples at 70.0 eps, loss = 3.808
    [batch 407]: seen 40700 examples at 70.0 eps, loss = 3.809
    [batch 415]: seen 41500 examples at 70.0 eps, loss = 3.810
    [batch 422]: seen 42200 examples at 70.0 eps, loss = 3.809
    [batch 429]: seen 42900 examples at 70.0 eps, loss = 3.804
    [batch 435]: seen 43500 examples at 69.7 eps, loss = 3.802
    [batch 438]: seen 43800 examples at 68.9 eps, loss = 3.800
    [batch 446]: seen 44600 examples at 69.0 eps, loss = 3.803
    [batch 454]: seen 45400 examples at 69.0 eps, loss = 3.806
    [batch 461]: seen 46100 examples at 69.0 eps, loss = 3.805
    [batch 469]: seen 46900 examples at 69.0 eps, loss = 3.802
    [batch 474]: seen 47400 examples at 68.7 eps, loss 

    [batch 1266]: seen 126600 examples at 65.6 eps, loss = 3.775
    [batch 1269]: seen 126900 examples at 65.3 eps, loss = 3.772
    [batch 1274]: seen 127400 examples at 65.2 eps, loss = 3.774
    [batch 1282]: seen 128200 examples at 65.2 eps, loss = 3.775
    [batch 1290]: seen 129000 examples at 65.2 eps, loss = 3.774
    [batch 1293]: seen 129300 examples at 65.0 eps, loss = 3.770
    [batch 1295]: seen 129500 examples at 64.8 eps, loss = 3.768
    [batch 1302]: seen 130200 examples at 64.8 eps, loss = 3.773
    [batch 1310]: seen 131000 examples at 64.9 eps, loss = 3.775
    [batch 1318]: seen 131800 examples at 64.9 eps, loss = 3.775
    [batch 1325]: seen 132500 examples at 64.9 eps, loss = 3.773
    [batch 1333]: seen 133300 examples at 64.9 eps, loss = 3.776
    [batch 1340]: seen 134000 examples at 65.0 eps, loss = 3.780
    [batch 1347]: seen 134700 examples at 65.0 eps, loss = 3.775
    [batch 1355]: seen 135500 examples at 65.0 eps, loss = 3.779
    [batch 1363]: seen 13

INFO:tensorflow:Restoring parameters from /home/ubuntu/W266/final_0/W266_Final/model_4/saved/train/model-15114
    [batch 1433]: seen 143300 examples at 64.9 eps, loss = 3.774
    [batch 1439]: seen 143900 examples at 64.9 eps, loss = 3.768
    [batch 1447]: seen 144700 examples at 64.9 eps, loss = 3.770
    [batch 1454]: seen 145400 examples at 64.9 eps, loss = 3.770
    [batch 1461]: seen 146100 examples at 64.9 eps, loss = 3.769
    [batch 1466]: seen 146600 examples at 64.9 eps, loss = 3.769
    [batch 1474]: seen 147400 examples at 64.9 eps, loss = 3.771
    [batch 1481]: seen 148100 examples at 64.9 eps, loss = 3.773
    [batch 1489]: seen 148900 examples at 64.9 eps, loss = 3.776
    [batch 1496]: seen 149600 examples at 65.0 eps, loss = 3.777
    [batch 1504]: seen 150400 examples at 65.0 eps, loss = 3.777
    [batch 1512]: seen 151200 examples at 65.0 eps, loss = 3.776
    [batch 1520]: seen 152000 examples at 65.0 eps, loss = 3.776
    [batch 1527]: seen 152700 examples at 65

# Train session 3

In [6]:
avg_loss = 3.7680904
best_loss = 3.7568204
curr_best = best_loss
train_step = 15398
epochs = 2
restore = True
epoch_start = 10

train_continue(hps,epochs,train_step,curr_best,best_loss,avg_loss,restore,epoch_start)







max_size of vocab was specified as 50000; we now have 50000 words. Stopping reading.
Finished constructing vocabulary of 50000 total words. Last word added: 16:03
INFO:tensorflow:Fetching data..
INFO:tensorflow:Creating batches..
INFO:tensorflow:[TOTAL Batches]  : 1872
INFO:tensorflow:[TOTAL Examples] : 187193
INFO:tensorflow:Creating batches..COMPLETE
INFO:tensorflow:Building core graph...
INFO:tensorflow:Adding attention_decoder timestep 0 of 100
INFO:tensorflow:Adding attention_decoder timestep 1 of 100
INFO:tensorflow:Adding attention_decoder timestep 2 of 100
INFO:tensorflow:Adding attention_decoder timestep 3 of 100
INFO:tensorflow:Adding attention_decoder timestep 4 of 100
INFO:tensorflow:Adding attention_decoder timestep 5 of 100
INFO:tensorflow:Adding attention_decoder timestep 6 of 100
INFO:tensorflow:Adding attention_decoder timestep 7 of 100
INFO:tensorflow:Adding attention_decoder timestep 8 of 100
INFO:tensorflow:Adding attention_decoder timestep 9 of 100
INFO:tenso

    [batch 85]: seen 8500 examples at 56.6 eps, loss = 3.763
    [batch 92]: seen 9200 examples at 57.3 eps, loss = 3.764
    [batch 99]: seen 9900 examples at 58.0 eps, loss = 3.761
    [batch 106]: seen 10600 examples at 58.6 eps, loss = 3.758
    [batch 113]: seen 11300 examples at 58.0 eps, loss = 3.757
    [batch 120]: seen 12000 examples at 58.5 eps, loss = 3.758
    [batch 127]: seen 12700 examples at 59.0 eps, loss = 3.766
    [batch 134]: seen 13400 examples at 59.4 eps, loss = 3.762
    [batch 141]: seen 14100 examples at 59.8 eps, loss = 3.763
    [batch 148]: seen 14800 examples at 60.2 eps, loss = 3.763
    [batch 155]: seen 15500 examples at 60.6 eps, loss = 3.758
    [batch 162]: seen 16200 examples at 60.9 eps, loss = 3.759
    [batch 169]: seen 16900 examples at 61.2 eps, loss = 3.758
    [batch 176]: seen 17600 examples at 61.4 eps, loss = 3.759
    [batch 182]: seen 18200 examples at 60.9 eps, loss = 3.756
    [batch 189]: seen 18900 examples at 61.2 eps, loss = 3.75

    [batch 924]: seen 92400 examples at 60.4 eps, loss = 3.749
    [batch 931]: seen 93100 examples at 60.4 eps, loss = 3.750
    [batch 938]: seen 93800 examples at 60.5 eps, loss = 3.752
    [batch 945]: seen 94500 examples at 60.6 eps, loss = 3.753
    [batch 952]: seen 95200 examples at 60.6 eps, loss = 3.759
    [batch 959]: seen 95900 examples at 60.7 eps, loss = 3.759
    [batch 966]: seen 96600 examples at 60.7 eps, loss = 3.758
    [batch 973]: seen 97300 examples at 60.8 eps, loss = 3.759
    [batch 980]: seen 98000 examples at 60.8 eps, loss = 3.764
    [batch 987]: seen 98700 examples at 60.9 eps, loss = 3.760
    [batch 994]: seen 99400 examples at 60.9 eps, loss = 3.763
    [batch 1001]: seen 100100 examples at 61.0 eps, loss = 3.755
    [batch 1008]: seen 100800 examples at 61.0 eps, loss = 3.749
    [batch 1015]: seen 101500 examples at 61.1 eps, loss = 3.751
    [batch 1022]: seen 102200 examples at 61.1 eps, loss = 3.746
    [batch 1029]: seen 102900 examples at 61.2 

    [batch 1770]: seen 177000 examples at 63.6 eps, loss = 3.739
    [batch 1777]: seen 177700 examples at 63.6 eps, loss = 3.738
    [batch 1784]: seen 178400 examples at 63.7 eps, loss = 3.738
    [batch 1791]: seen 179100 examples at 63.7 eps, loss = 3.737
    [batch 1798]: seen 179800 examples at 63.7 eps, loss = 3.737
    [batch 1805]: seen 180500 examples at 63.7 eps, loss = 3.737
    [batch 1812]: seen 181200 examples at 63.7 eps, loss = 3.735
    [batch 1819]: seen 181900 examples at 63.8 eps, loss = 3.735
    [batch 1826]: seen 182600 examples at 63.8 eps, loss = 3.731
    [batch 1833]: seen 183300 examples at 63.8 eps, loss = 3.735
    [batch 1840]: seen 184000 examples at 63.8 eps, loss = 3.734
    [batch 1847]: seen 184700 examples at 63.8 eps, loss = 3.728
    [batch 1853]: seen 185300 examples at 63.8 eps, loss = 3.722
    [batch 1858]: seen 185800 examples at 63.7 eps, loss = 3.720
    [batch 1861]: seen 186100 examples at 63.5 eps, loss = 3.719
    [batch 1864]: seen 18

    [batch 746]: seen 74600 examples at 64.3 eps, loss = 3.699
    [batch 753]: seen 75300 examples at 64.4 eps, loss = 3.696
    [batch 760]: seen 76000 examples at 64.4 eps, loss = 3.697
    [batch 767]: seen 76700 examples at 64.5 eps, loss = 3.694
    [batch 774]: seen 77400 examples at 64.5 eps, loss = 3.693
    [batch 779]: seen 77900 examples at 64.4 eps, loss = 3.689
    [batch 786]: seen 78600 examples at 64.4 eps, loss = 3.691
    [batch 793]: seen 79300 examples at 64.5 eps, loss = 3.692
    [batch 800]: seen 80000 examples at 64.5 eps, loss = 3.691
    [batch 804]: seen 80400 examples at 64.2 eps, loss = 3.686
    [batch 807]: seen 80700 examples at 63.7 eps, loss = 3.683
    [batch 810]: seen 81000 examples at 63.4 eps, loss = 3.680
    [batch 813]: seen 81300 examples at 63.1 eps, loss = 3.680
    [batch 818]: seen 81800 examples at 62.9 eps, loss = 3.682
    [batch 825]: seen 82500 examples at 63.0 eps, loss = 3.688
    [batch 832]: seen 83200 examples at 63.0 eps, loss 

KeyboardInterrupt: 

# Train session 4

In [6]:
avg_loss = 3.694
best_loss = 3.694
curr_best = best_loss
train_step = 18059
epochs = 15
restore = True
epoch_start = 11

train_continue(hps,epochs,train_step,curr_best,best_loss,avg_loss,restore,epoch_start)







max_size of vocab was specified as 50000; we now have 50000 words. Stopping reading.
Finished constructing vocabulary of 50000 total words. Last word added: 16:03
INFO:tensorflow:Fetching data..
INFO:tensorflow:Creating batches..
INFO:tensorflow:[TOTAL Batches]  : 1872
INFO:tensorflow:[TOTAL Examples] : 187193
INFO:tensorflow:Creating batches..COMPLETE
INFO:tensorflow:Building core graph...
INFO:tensorflow:Adding attention_decoder timestep 0 of 100
INFO:tensorflow:Adding attention_decoder timestep 1 of 100
INFO:tensorflow:Adding attention_decoder timestep 2 of 100
INFO:tensorflow:Adding attention_decoder timestep 3 of 100
INFO:tensorflow:Adding attention_decoder timestep 4 of 100
INFO:tensorflow:Adding attention_decoder timestep 5 of 100
INFO:tensorflow:Adding attention_decoder timestep 6 of 100
INFO:tensorflow:Adding attention_decoder timestep 7 of 100
INFO:tensorflow:Adding attention_decoder timestep 8 of 100
INFO:tensorflow:Adding attention_decoder timestep 9 of 100
INFO:tenso

    [batch 49]: seen 4900 examples at 27.2 eps, loss = 3.671
    [batch 56]: seen 5600 examples at 29.4 eps, loss = 3.676
    [batch 63]: seen 6300 examples at 31.4 eps, loss = 3.676
    [batch 70]: seen 7000 examples at 33.2 eps, loss = 3.676
    [batch 77]: seen 7700 examples at 34.9 eps, loss = 3.673
    [batch 82]: seen 8200 examples at 35.4 eps, loss = 3.670
    [batch 85]: seen 8500 examples at 35.0 eps, loss = 3.669
    [batch 88]: seen 8800 examples at 34.2 eps, loss = 3.666
    [batch 95]: seen 9500 examples at 35.5 eps, loss = 3.671
    [batch 102]: seen 10200 examples at 36.7 eps, loss = 3.676
    [batch 109]: seen 10900 examples at 37.9 eps, loss = 3.675
    [batch 116]: seen 11600 examples at 38.9 eps, loss = 3.674
    [batch 123]: seen 12300 examples at 39.9 eps, loss = 3.678
    [batch 130]: seen 13000 examples at 40.8 eps, loss = 3.681
    [EXCEPTION]:  Loss is not finite. ; Restoring model params
INFO:tensorflow:Loading checkpoint /home/ubuntu/W266/final_0/W266_Final/m

    [batch 911]: seen 91100 examples at 62.2 eps, loss = 3.668
    [batch 918]: seen 91800 examples at 62.3 eps, loss = 3.666
    [batch 925]: seen 92500 examples at 62.3 eps, loss = 3.666
    [batch 932]: seen 93200 examples at 62.3 eps, loss = 3.668
    [batch 939]: seen 93900 examples at 62.4 eps, loss = 3.672
    [batch 946]: seen 94600 examples at 62.4 eps, loss = 3.674
    [batch 953]: seen 95300 examples at 62.5 eps, loss = 3.674
    [batch 960]: seen 96000 examples at 62.5 eps, loss = 3.676
    [batch 967]: seen 96700 examples at 62.6 eps, loss = 3.674
    [batch 974]: seen 97400 examples at 62.6 eps, loss = 3.676
    [batch 981]: seen 98100 examples at 62.7 eps, loss = 3.675
    [batch 988]: seen 98800 examples at 62.7 eps, loss = 3.677
    [batch 995]: seen 99500 examples at 62.8 eps, loss = 3.679
    [batch 1002]: seen 100200 examples at 62.8 eps, loss = 3.679
    [batch 1009]: seen 100900 examples at 62.8 eps, loss = 3.676
    [batch 1016]: seen 101600 examples at 62.9 eps,

    [batch 1774]: seen 177400 examples at 64.0 eps, loss = 3.656
    [batch 1782]: seen 178200 examples at 64.0 eps, loss = 3.662
    [batch 1789]: seen 178900 examples at 64.0 eps, loss = 3.669
    [batch 1796]: seen 179600 examples at 64.0 eps, loss = 3.668
    [batch 1803]: seen 180300 examples at 64.1 eps, loss = 3.667
    [batch 1810]: seen 181000 examples at 64.1 eps, loss = 3.667
    [batch 1817]: seen 181700 examples at 64.1 eps, loss = 3.664
    [batch 1824]: seen 182400 examples at 64.1 eps, loss = 3.668
    [batch 1831]: seen 183100 examples at 64.1 eps, loss = 3.663
    [batch 1838]: seen 183800 examples at 64.2 eps, loss = 3.658
    [batch 1845]: seen 184500 examples at 64.2 eps, loss = 3.657
    [batch 1853]: seen 185300 examples at 64.2 eps, loss = 3.653
    [batch 1860]: seen 186000 examples at 64.2 eps, loss = 3.652
    [batch 1867]: seen 186700 examples at 64.2 eps, loss = 3.651
    [END] Training complete: Total examples : 187100; Total time: 0:48:31
[EPOCH 12] Compl

    [batch 777]: seen 77700 examples at 64.6 eps, loss = 3.636
    [batch 784]: seen 78400 examples at 64.7 eps, loss = 3.639
    [batch 791]: seen 79100 examples at 64.7 eps, loss = 3.642
    [batch 798]: seen 79800 examples at 64.8 eps, loss = 3.643
    [batch 805]: seen 80500 examples at 64.8 eps, loss = 3.640
    [batch 812]: seen 81200 examples at 64.8 eps, loss = 3.640
    [batch 820]: seen 82000 examples at 64.9 eps, loss = 3.640
    [batch 827]: seen 82700 examples at 64.9 eps, loss = 3.638
    [batch 834]: seen 83400 examples at 65.0 eps, loss = 3.638
    [batch 841]: seen 84100 examples at 65.0 eps, loss = 3.639
    [batch 848]: seen 84800 examples at 65.0 eps, loss = 3.642
    [batch 855]: seen 85500 examples at 65.1 eps, loss = 3.641
    [batch 862]: seen 86200 examples at 65.1 eps, loss = 3.644
    [batch 869]: seen 86900 examples at 65.1 eps, loss = 3.643
    [batch 876]: seen 87600 examples at 65.2 eps, loss = 3.632
    [batch 884]: seen 88400 examples at 65.2 eps, loss 

...

    [batch 492]: seen 49200 examples at 64.3 eps, loss = 3.560
    [batch 499]: seen 49900 examples at 64.3 eps, loss = 3.563
    [batch 507]: seen 50700 examples at 64.4 eps, loss = 3.566
    [batch 514]: seen 51400 examples at 64.5 eps, loss = 3.566
    [batch 521]: seen 52100 examples at 64.5 eps, loss = 3.564
    [batch 528]: seen 52800 examples at 64.6 eps, loss = 3.569
    [batch 535]: seen 53500 examples at 64.7 eps, loss = 3.570
    [batch 542]: seen 54200 examples at 64.7 eps, loss = 3.572
    [batch 549]: seen 54900 examples at 64.8 eps, loss = 3.573
    [batch 557]: seen 55700 examples at 64.9 eps, loss = 3.572
    [batch 564]: seen 56400 examples at 64.9 eps, loss = 3.570
    [batch 571]: seen 57100 examples at 65.0 eps, loss = 3.574
    [batch 579]: seen 57900 examples at 65.0 eps, loss = 3.572
    [batch 586]: seen 58600 examples at 65.1 eps, loss = 3.569
    [batch 593]: seen 59300 examples at 65.1 eps, loss = 3.570
    [batch 600]: seen 60000 examples at 65.2 eps, loss 

    [batch 1367]: seen 136700 examples at 64.8 eps, loss = 3.561
    [batch 1374]: seen 137400 examples at 64.9 eps, loss = 3.557
    [batch 1382]: seen 138200 examples at 64.9 eps, loss = 3.555
    [batch 1389]: seen 138900 examples at 64.9 eps, loss = 3.556
    [batch 1396]: seen 139600 examples at 64.9 eps, loss = 3.555
    [batch 1403]: seen 140300 examples at 64.9 eps, loss = 3.555
    [batch 1410]: seen 141000 examples at 65.0 eps, loss = 3.559
    [batch 1417]: seen 141700 examples at 65.0 eps, loss = 3.555
    [batch 1424]: seen 142400 examples at 65.0 eps, loss = 3.554
    [batch 1431]: seen 143100 examples at 65.0 eps, loss = 3.553
    [batch 1438]: seen 143800 examples at 65.1 eps, loss = 3.550
    [batch 1445]: seen 144500 examples at 65.1 eps, loss = 3.552
    [batch 1452]: seen 145200 examples at 65.1 eps, loss = 3.551
    [batch 1459]: seen 145900 examples at 65.1 eps, loss = 3.550
    [batch 1466]: seen 146600 examples at 65.1 eps, loss = 3.547
    [batch 1474]: seen 14

    [batch 378]: seen 37800 examples at 67.0 eps, loss = 3.536
    [batch 385]: seen 38500 examples at 67.1 eps, loss = 3.536
    [batch 392]: seen 39200 examples at 67.1 eps, loss = 3.534
    [batch 399]: seen 39900 examples at 67.2 eps, loss = 3.534
    [batch 406]: seen 40600 examples at 67.2 eps, loss = 3.535
    [batch 413]: seen 41300 examples at 67.2 eps, loss = 3.533
    [batch 421]: seen 42100 examples at 67.3 eps, loss = 3.533
    [batch 428]: seen 42800 examples at 67.3 eps, loss = 3.530
    [batch 435]: seen 43500 examples at 67.4 eps, loss = 3.530
    [batch 443]: seen 44300 examples at 67.4 eps, loss = 3.529
    [batch 450]: seen 45000 examples at 67.5 eps, loss = 3.530
    [batch 455]: seen 45500 examples at 67.2 eps, loss = 3.524
    [batch 462]: seen 46200 examples at 67.2 eps, loss = 3.526
    [batch 464]: seen 46400 examples at 66.5 eps, loss = 3.522
    [batch 467]: seen 46700 examples at 65.6 eps, loss = 3.520
    [batch 472]: seen 47200 examples at 65.0 eps, loss 

    [batch 1236]: seen 123600 examples at 66.4 eps, loss = 3.520
    [batch 1243]: seen 124300 examples at 66.4 eps, loss = 3.525
    [batch 1250]: seen 125000 examples at 66.4 eps, loss = 3.529
    [batch 1257]: seen 125700 examples at 66.5 eps, loss = 3.527
    [batch 1264]: seen 126400 examples at 66.5 eps, loss = 3.533
    [batch 1271]: seen 127100 examples at 66.5 eps, loss = 3.534
    [batch 1278]: seen 127800 examples at 66.5 eps, loss = 3.536
    [batch 1285]: seen 128500 examples at 66.5 eps, loss = 3.530
    [batch 1293]: seen 129300 examples at 66.6 eps, loss = 3.533
    [batch 1300]: seen 130000 examples at 66.6 eps, loss = 3.536
    [batch 1307]: seen 130700 examples at 66.6 eps, loss = 3.537
    [batch 1314]: seen 131400 examples at 66.6 eps, loss = 3.531
    [batch 1321]: seen 132100 examples at 66.6 eps, loss = 3.532
    [batch 1329]: seen 132900 examples at 66.6 eps, loss = 3.530
    [batch 1336]: seen 133600 examples at 66.7 eps, loss = 3.528
    [batch 1343]: seen 13

KeyboardInterrupt: 

# Train session 5

In [6]:
avg_loss = 3.6
best_loss = 3.6
curr_best = best_loss
train_step = 26879
epochs = 6
restore = True
epoch_start = 16

train_continue(hps,epochs,train_step,curr_best,best_loss,avg_loss,restore,epoch_start)







max_size of vocab was specified as 50000; we now have 50000 words. Stopping reading.
Finished constructing vocabulary of 50000 total words. Last word added: 16:03
INFO:tensorflow:Fetching data..
INFO:tensorflow:Creating batches..
INFO:tensorflow:[TOTAL Batches]  : 2808
INFO:tensorflow:[TOTAL Examples] : 280778
INFO:tensorflow:Creating batches..COMPLETE
INFO:tensorflow:Building core graph...
INFO:tensorflow:Adding attention_decoder timestep 0 of 100
INFO:tensorflow:Adding attention_decoder timestep 1 of 100
INFO:tensorflow:Adding attention_decoder timestep 2 of 100
INFO:tensorflow:Adding attention_decoder timestep 3 of 100
INFO:tensorflow:Adding attention_decoder timestep 4 of 100
INFO:tensorflow:Adding attention_decoder timestep 5 of 100
INFO:tensorflow:Adding attention_decoder timestep 6 of 100
INFO:tensorflow:Adding attention_decoder timestep 7 of 100
INFO:tensorflow:Adding attention_decoder timestep 8 of 100
INFO:tensorflow:Adding attention_decoder timestep 9 of 100
INFO:tenso

    [batch 17]: seen 1700 examples : 16.2 eps, Loss: 3.487, Avg loss: 3.581, Best loss: 3.581, cov loss: 0.189
    [batch 20]: seen 2000 examples : 17.2 eps, Loss: 3.428, Avg loss: 3.577, Best loss: 3.577, cov loss: 0.170
    [batch 24]: seen 2400 examples : 18.4 eps, Loss: 3.412, Avg loss: 3.575, Best loss: 3.575, cov loss: 0.200
    [batch 26]: seen 2600 examples : 18.5 eps, Loss: 3.556, Avg loss: 3.574, Best loss: 3.574, cov loss: 0.188
    [batch 33]: seen 3300 examples : 21.9 eps, Loss: 3.484, Avg loss: 3.575, Best loss: 3.574, cov loss: 0.178
    [batch 40]: seen 4000 examples : 24.3 eps, Loss: 3.362, Avg loss: 3.572, Best loss: 3.572, cov loss: 0.173
    [batch 47]: seen 4700 examples : 26.9 eps, Loss: 3.615, Avg loss: 3.574, Best loss: 3.572, cov loss: 0.187
    [batch 52]: seen 5200 examples : 28.0 eps, Loss: 3.793, Avg loss: 3.574, Best loss: 3.571, cov loss: 0.198
    [batch 59]: seen 5900 examples : 30.1 eps, Loss: 3.614, Avg loss: 3.577, Best loss: 3.571, cov loss: 0.190
 

    [batch 414]: seen 41400 examples : 43.9 eps, Loss: 3.332, Avg loss: 3.519, Best loss: 3.514, cov loss: 0.165
    [batch 421]: seen 42100 examples : 44.2 eps, Loss: 3.475, Avg loss: 3.519, Best loss: 3.514, cov loss: 0.178
    [batch 428]: seen 42800 examples : 44.4 eps, Loss: 3.366, Avg loss: 3.516, Best loss: 3.514, cov loss: 0.193
    [batch 435]: seen 43500 examples : 44.7 eps, Loss: 3.223, Avg loss: 3.516, Best loss: 3.514, cov loss: 0.168
    [batch 440]: seen 44000 examples : 44.7 eps, Loss: 3.651, Avg loss: 3.515, Best loss: 3.513, cov loss: 0.190
    [batch 443]: seen 44300 examples : 44.5 eps, Loss: 3.455, Avg loss: 3.512, Best loss: 3.512, cov loss: 0.185
    [batch 445]: seen 44500 examples : 44.2 eps, Loss: 3.280, Avg loss: 3.508, Best loss: 3.508, cov loss: 0.178
    [batch 449]: seen 44900 examples : 44.2 eps, Loss: 3.725, Avg loss: 3.512, Best loss: 3.508, cov loss: 0.179
    [batch 456]: seen 45600 examples : 44.4 eps, Loss: 3.587, Avg loss: 3.515, Best loss: 3.508,

    [batch 903]: seen 90300 examples : 53.2 eps, Loss: 3.558, Avg loss: 3.520, Best loss: 3.503, cov loss: 0.185
    [batch 910]: seen 91000 examples : 53.3 eps, Loss: 3.514, Avg loss: 3.521, Best loss: 3.503, cov loss: 0.166
    [batch 917]: seen 91700 examples : 53.4 eps, Loss: 3.786, Avg loss: 3.520, Best loss: 3.503, cov loss: 0.189
    [batch 924]: seen 92400 examples : 53.5 eps, Loss: 3.417, Avg loss: 3.522, Best loss: 3.503, cov loss: 0.191
    [batch 931]: seen 93100 examples : 53.6 eps, Loss: 3.574, Avg loss: 3.524, Best loss: 3.503, cov loss: 0.189
    [batch 938]: seen 93800 examples : 53.6 eps, Loss: 3.578, Avg loss: 3.525, Best loss: 3.503, cov loss: 0.176
    [batch 945]: seen 94500 examples : 53.7 eps, Loss: 3.574, Avg loss: 3.520, Best loss: 3.503, cov loss: 0.190
    [batch 952]: seen 95200 examples : 53.8 eps, Loss: 3.553, Avg loss: 3.518, Best loss: 3.503, cov loss: 0.177
    [batch 959]: seen 95900 examples : 53.9 eps, Loss: 3.761, Avg loss: 3.514, Best loss: 3.503,

    [batch 1399]: seen 139900 examples : 57.4 eps, Loss: 3.533, Avg loss: 3.512, Best loss: 3.501, cov loss: 0.175
    [batch 1406]: seen 140600 examples : 57.4 eps, Loss: 3.446, Avg loss: 3.513, Best loss: 3.501, cov loss: 0.193
    [batch 1413]: seen 141300 examples : 57.5 eps, Loss: 3.791, Avg loss: 3.512, Best loss: 3.501, cov loss: 0.189
    [batch 1420]: seen 142000 examples : 57.5 eps, Loss: 3.465, Avg loss: 3.511, Best loss: 3.501, cov loss: 0.163
    [batch 1427]: seen 142700 examples : 57.6 eps, Loss: 3.645, Avg loss: 3.516, Best loss: 3.501, cov loss: 0.185
    [batch 1434]: seen 143400 examples : 57.6 eps, Loss: 3.363, Avg loss: 3.511, Best loss: 3.501, cov loss: 0.175
    [batch 1441]: seen 144100 examples : 57.6 eps, Loss: 3.532, Avg loss: 3.515, Best loss: 3.501, cov loss: 0.184
    [batch 1448]: seen 144800 examples : 57.7 eps, Loss: 3.479, Avg loss: 3.510, Best loss: 3.501, cov loss: 0.188
    [batch 1455]: seen 145500 examples : 57.7 eps, Loss: 3.556, Avg loss: 3.511,

...

    [batch 597]: seen 59700 examples : 61.5 eps, Loss: 3.247, Avg loss: 3.346, Best loss: 3.346, cov loss: 0.151
    [batch 602]: seen 60200 examples : 61.4 eps, Loss: 3.175, Avg loss: 3.347, Best loss: 3.346, cov loss: 0.169
    [batch 607]: seen 60700 examples : 61.2 eps, Loss: 3.382, Avg loss: 3.347, Best loss: 3.346, cov loss: 0.175
    [batch 614]: seen 61400 examples : 61.3 eps, Loss: 3.392, Avg loss: 3.349, Best loss: 3.346, cov loss: 0.171
    [batch 620]: seen 62000 examples : 61.1 eps, Loss: 3.151, Avg loss: 3.345, Best loss: 3.345, cov loss: 0.155
    [batch 625]: seen 62500 examples : 61.0 eps, Loss: 3.286, Avg loss: 3.345, Best loss: 3.344, cov loss: 0.167
    [batch 629]: seen 62900 examples : 60.6 eps, Loss: 3.278, Avg loss: 3.343, Best loss: 3.343, cov loss: 0.151
    [batch 633]: seen 63300 examples : 60.4 eps, Loss: 3.313, Avg loss: 3.343, Best loss: 3.342, cov loss: 0.163
    [batch 638]: seen 63800 examples : 60.3 eps, Loss: 3.260, Avg loss: 3.342, Best loss: 3.340,

    [batch 1086]: seen 108600 examples : 63.1 eps, Loss: 3.342, Avg loss: 3.363, Best loss: 3.338, cov loss: 0.149
    [batch 1093]: seen 109300 examples : 63.1 eps, Loss: 3.559, Avg loss: 3.365, Best loss: 3.338, cov loss: 0.179
    [batch 1100]: seen 110000 examples : 63.2 eps, Loss: 3.385, Avg loss: 3.368, Best loss: 3.338, cov loss: 0.174
    [batch 1107]: seen 110700 examples : 63.2 eps, Loss: 3.473, Avg loss: 3.369, Best loss: 3.338, cov loss: 0.177
    [batch 1114]: seen 111400 examples : 63.2 eps, Loss: 3.384, Avg loss: 3.373, Best loss: 3.338, cov loss: 0.183
    [batch 1121]: seen 112100 examples : 63.3 eps, Loss: 3.364, Avg loss: 3.373, Best loss: 3.338, cov loss: 0.155
    [batch 1128]: seen 112800 examples : 63.3 eps, Loss: 3.334, Avg loss: 3.372, Best loss: 3.338, cov loss: 0.153
    [batch 1135]: seen 113500 examples : 63.3 eps, Loss: 3.343, Avg loss: 3.374, Best loss: 3.338, cov loss: 0.142
    [batch 1142]: seen 114200 examples : 63.4 eps, Loss: 3.278, Avg loss: 3.375,

    [batch 1590]: seen 159000 examples : 64.9 eps, Loss: 3.395, Avg loss: 3.357, Best loss: 3.338, cov loss: 0.169
    [batch 1597]: seen 159700 examples : 64.9 eps, Loss: 3.399, Avg loss: 3.360, Best loss: 3.338, cov loss: 0.175
    [batch 1604]: seen 160400 examples : 64.9 eps, Loss: 3.439, Avg loss: 3.366, Best loss: 3.338, cov loss: 0.164
    [batch 1611]: seen 161100 examples : 64.9 eps, Loss: 3.338, Avg loss: 3.367, Best loss: 3.338, cov loss: 0.169
    [batch 1618]: seen 161800 examples : 64.9 eps, Loss: 3.152, Avg loss: 3.370, Best loss: 3.338, cov loss: 0.152
    [batch 1625]: seen 162500 examples : 65.0 eps, Loss: 3.300, Avg loss: 3.370, Best loss: 3.338, cov loss: 0.175
    [batch 1632]: seen 163200 examples : 65.0 eps, Loss: 3.549, Avg loss: 3.373, Best loss: 3.338, cov loss: 0.187
    [batch 1639]: seen 163900 examples : 65.0 eps, Loss: 3.527, Avg loss: 3.374, Best loss: 3.338, cov loss: 0.172
    [batch 1646]: seen 164600 examples : 65.0 eps, Loss: 3.352, Avg loss: 3.374,

    [batch 2094]: seen 209400 examples : 65.8 eps, Loss: 3.485, Avg loss: 3.365, Best loss: 3.338, cov loss: 0.173
    [batch 2101]: seen 210100 examples : 65.8 eps, Loss: 3.275, Avg loss: 3.362, Best loss: 3.338, cov loss: 0.156
    [batch 2108]: seen 210800 examples : 65.8 eps, Loss: 3.572, Avg loss: 3.362, Best loss: 3.338, cov loss: 0.170
    [batch 2115]: seen 211500 examples : 65.8 eps, Loss: 3.418, Avg loss: 3.361, Best loss: 3.338, cov loss: 0.170
    [batch 2122]: seen 212200 examples : 65.9 eps, Loss: 3.296, Avg loss: 3.364, Best loss: 3.338, cov loss: 0.181
    [batch 2129]: seen 212900 examples : 65.9 eps, Loss: 3.317, Avg loss: 3.366, Best loss: 3.338, cov loss: 0.156
    [batch 2136]: seen 213600 examples : 65.9 eps, Loss: 3.321, Avg loss: 3.367, Best loss: 3.338, cov loss: 0.159
    [batch 2143]: seen 214300 examples : 65.9 eps, Loss: 3.246, Avg loss: 3.365, Best loss: 3.338, cov loss: 0.161
    [batch 2150]: seen 215000 examples : 65.9 eps, Loss: 3.352, Avg loss: 3.362,

    [batch 2576]: seen 257600 examples : 66.4 eps, Loss: 3.417, Avg loss: 3.371, Best loss: 3.338, cov loss: 0.182
    [batch 2583]: seen 258300 examples : 66.4 eps, Loss: 3.377, Avg loss: 3.372, Best loss: 3.338, cov loss: 0.164
    [batch 2590]: seen 259000 examples : 66.4 eps, Loss: 3.385, Avg loss: 3.366, Best loss: 3.338, cov loss: 0.160
    [batch 2597]: seen 259700 examples : 66.4 eps, Loss: 3.221, Avg loss: 3.364, Best loss: 3.338, cov loss: 0.160
    [batch 2604]: seen 260400 examples : 66.4 eps, Loss: 3.405, Avg loss: 3.371, Best loss: 3.338, cov loss: 0.190
    [batch 2611]: seen 261100 examples : 66.4 eps, Loss: 3.584, Avg loss: 3.373, Best loss: 3.338, cov loss: 0.170
    [batch 2618]: seen 261800 examples : 66.4 eps, Loss: 3.259, Avg loss: 3.372, Best loss: 3.338, cov loss: 0.161
    [batch 2625]: seen 262500 examples : 66.4 eps, Loss: 3.371, Avg loss: 3.371, Best loss: 3.338, cov loss: 0.164
    [batch 2632]: seen 263200 examples : 66.4 eps, Loss: 3.328, Avg loss: 3.372,

# Train session 6

In [6]:
avg_loss = 3.371
best_loss = 3.337
curr_best = best_loss
train_step = 38874
epochs = 3
restore = True
epoch_start = 22

train_continue(hps,epochs,train_step,curr_best,best_loss,avg_loss,restore,epoch_start)







max_size of vocab was specified as 50000; we now have 50000 words. Stopping reading.
Finished constructing vocabulary of 50000 total words. Last word added: 16:03
INFO:tensorflow:Fetching data..
INFO:tensorflow:Creating batches..
INFO:tensorflow:[TOTAL Batches]  : 2808
INFO:tensorflow:[TOTAL Examples] : 280778
INFO:tensorflow:Creating batches..COMPLETE
INFO:tensorflow:Building core graph...
INFO:tensorflow:Adding attention_decoder timestep 0 of 100
INFO:tensorflow:Adding attention_decoder timestep 1 of 100
INFO:tensorflow:Adding attention_decoder timestep 2 of 100
INFO:tensorflow:Adding attention_decoder timestep 3 of 100
INFO:tensorflow:Adding attention_decoder timestep 4 of 100
INFO:tensorflow:Adding attention_decoder timestep 5 of 100
INFO:tensorflow:Adding attention_decoder timestep 6 of 100
INFO:tensorflow:Adding attention_decoder timestep 7 of 100
INFO:tensorflow:Adding attention_decoder timestep 8 of 100
INFO:tensorflow:Adding attention_decoder timestep 9 of 100
INFO:tenso

    [batch 50]: seen 5000 examples : 49.5 eps, Loss: 3.443, Avg loss: 3.371, Best loss: 3.337, cov loss: 0.175
    [batch 57]: seen 5700 examples : 51.2 eps, Loss: 3.474, Avg loss: 3.373, Best loss: 3.337, cov loss: 0.181
    [batch 64]: seen 6400 examples : 52.7 eps, Loss: 3.348, Avg loss: 3.377, Best loss: 3.337, cov loss: 0.158
    [batch 71]: seen 7100 examples : 53.9 eps, Loss: 3.364, Avg loss: 3.374, Best loss: 3.337, cov loss: 0.168
    [batch 78]: seen 7800 examples : 55.0 eps, Loss: 3.547, Avg loss: 3.374, Best loss: 3.337, cov loss: 0.162
    [batch 85]: seen 8500 examples : 55.9 eps, Loss: 3.284, Avg loss: 3.376, Best loss: 3.337, cov loss: 0.168
    [batch 92]: seen 9200 examples : 56.7 eps, Loss: 3.320, Avg loss: 3.376, Best loss: 3.337, cov loss: 0.166
    [batch 99]: seen 9900 examples : 57.4 eps, Loss: 3.505, Avg loss: 3.377, Best loss: 3.337, cov loss: 0.176
    [batch 106]: seen 10600 examples : 58.0 eps, Loss: 3.357, Avg loss: 3.377, Best loss: 3.337, cov loss: 0.176

    [batch 561]: seen 56100 examples : 66.4 eps, Loss: 3.690, Avg loss: 3.361, Best loss: 3.337, cov loss: 0.186
    [batch 568]: seen 56800 examples : 66.4 eps, Loss: 3.458, Avg loss: 3.363, Best loss: 3.337, cov loss: 0.176
    [batch 575]: seen 57500 examples : 66.4 eps, Loss: 3.292, Avg loss: 3.360, Best loss: 3.337, cov loss: 0.178
    [batch 582]: seen 58200 examples : 66.5 eps, Loss: 3.286, Avg loss: 3.361, Best loss: 3.337, cov loss: 0.150
    [batch 589]: seen 58900 examples : 66.5 eps, Loss: 3.353, Avg loss: 3.359, Best loss: 3.337, cov loss: 0.173
    [batch 596]: seen 59600 examples : 66.5 eps, Loss: 3.601, Avg loss: 3.357, Best loss: 3.337, cov loss: 0.177
    [batch 603]: seen 60300 examples : 66.5 eps, Loss: 3.497, Avg loss: 3.354, Best loss: 3.337, cov loss: 0.171
    [batch 610]: seen 61000 examples : 66.6 eps, Loss: 3.341, Avg loss: 3.357, Best loss: 3.337, cov loss: 0.171
    [batch 617]: seen 61700 examples : 66.6 eps, Loss: 3.290, Avg loss: 3.353, Best loss: 3.337,

    [batch 1072]: seen 107200 examples : 67.5 eps, Loss: 3.390, Avg loss: 3.364, Best loss: 3.337, cov loss: 0.181
    [batch 1079]: seen 107900 examples : 67.5 eps, Loss: 3.328, Avg loss: 3.368, Best loss: 3.337, cov loss: 0.163
    [batch 1086]: seen 108600 examples : 67.6 eps, Loss: 3.336, Avg loss: 3.365, Best loss: 3.337, cov loss: 0.172
    [batch 1093]: seen 109300 examples : 67.6 eps, Loss: 3.418, Avg loss: 3.364, Best loss: 3.337, cov loss: 0.168
    [batch 1100]: seen 110000 examples : 67.6 eps, Loss: 3.531, Avg loss: 3.362, Best loss: 3.337, cov loss: 0.186
    [batch 1107]: seen 110700 examples : 67.6 eps, Loss: 3.394, Avg loss: 3.360, Best loss: 3.337, cov loss: 0.188
    [batch 1114]: seen 111400 examples : 67.6 eps, Loss: 3.194, Avg loss: 3.360, Best loss: 3.337, cov loss: 0.153
    [batch 1121]: seen 112100 examples : 67.6 eps, Loss: 3.441, Avg loss: 3.358, Best loss: 3.337, cov loss: 0.166
    [batch 1128]: seen 112800 examples : 67.6 eps, Loss: 3.285, Avg loss: 3.360,

    [batch 1569]: seen 156900 examples : 67.7 eps, Loss: 3.411, Avg loss: 3.348, Best loss: 3.336, cov loss: 0.180
    [batch 1576]: seen 157600 examples : 67.7 eps, Loss: 3.357, Avg loss: 3.349, Best loss: 3.336, cov loss: 0.167
    [batch 1583]: seen 158300 examples : 67.7 eps, Loss: 3.334, Avg loss: 3.348, Best loss: 3.336, cov loss: 0.177
    [batch 1590]: seen 159000 examples : 67.7 eps, Loss: 3.342, Avg loss: 3.347, Best loss: 3.336, cov loss: 0.162
    [batch 1597]: seen 159700 examples : 67.7 eps, Loss: 3.576, Avg loss: 3.352, Best loss: 3.336, cov loss: 0.173
    [batch 1604]: seen 160400 examples : 67.7 eps, Loss: 3.175, Avg loss: 3.349, Best loss: 3.336, cov loss: 0.170
    [batch 1611]: seen 161100 examples : 67.7 eps, Loss: 3.412, Avg loss: 3.352, Best loss: 3.336, cov loss: 0.162
    [batch 1618]: seen 161800 examples : 67.7 eps, Loss: 3.545, Avg loss: 3.357, Best loss: 3.336, cov loss: 0.169
    [batch 1625]: seen 162500 examples : 67.7 eps, Loss: 3.269, Avg loss: 3.358,

    [batch 2052]: seen 205200 examples : 67.9 eps, Loss: 3.484, Avg loss: 3.364, Best loss: 3.336, cov loss: 0.159
    [batch 2059]: seen 205900 examples : 67.9 eps, Loss: 3.272, Avg loss: 3.360, Best loss: 3.336, cov loss: 0.162
    [batch 2066]: seen 206600 examples : 68.0 eps, Loss: 3.349, Avg loss: 3.360, Best loss: 3.336, cov loss: 0.162
    [batch 2073]: seen 207300 examples : 68.0 eps, Loss: 3.441, Avg loss: 3.363, Best loss: 3.336, cov loss: 0.167
    [batch 2080]: seen 208000 examples : 68.0 eps, Loss: 3.403, Avg loss: 3.364, Best loss: 3.336, cov loss: 0.173
    [batch 2087]: seen 208700 examples : 68.0 eps, Loss: 3.321, Avg loss: 3.367, Best loss: 3.336, cov loss: 0.168
    [batch 2094]: seen 209400 examples : 68.0 eps, Loss: 3.444, Avg loss: 3.372, Best loss: 3.336, cov loss: 0.181
    [batch 2101]: seen 210100 examples : 68.0 eps, Loss: 3.627, Avg loss: 3.373, Best loss: 3.336, cov loss: 0.165
    [batch 2108]: seen 210800 examples : 68.0 eps, Loss: 3.487, Avg loss: 3.373,

KeyboardInterrupt: 

# Train session 7

In [7]:
avg_loss = 0
best_loss = None
curr_best = best_loss
train_step = 40416
epochs = 8
restore = True
epoch_start = 22

train_continue(hps,epochs,train_step,curr_best,best_loss,avg_loss,restore,epoch_start)







max_size of vocab was specified as 50000; we now have 50000 words. Stopping reading.
Finished constructing vocabulary of 50000 total words. Last word added: 16:03
INFO:tensorflow:Fetching data..
INFO:tensorflow:Creating batches..
INFO:tensorflow:[TOTAL Batches]  : 2808
INFO:tensorflow:[TOTAL Examples] : 280778
INFO:tensorflow:Creating batches..COMPLETE
INFO:tensorflow:Building core graph...
INFO:tensorflow:Adding attention_decoder timestep 0 of 100
INFO:tensorflow:Adding attention_decoder timestep 1 of 100
INFO:tensorflow:Adding attention_decoder timestep 2 of 100
INFO:tensorflow:Adding attention_decoder timestep 3 of 100
INFO:tensorflow:Adding attention_decoder timestep 4 of 100
INFO:tensorflow:Adding attention_decoder timestep 5 of 100
INFO:tensorflow:Adding attention_decoder timestep 6 of 100
INFO:tensorflow:Adding attention_decoder timestep 7 of 100
INFO:tensorflow:Adding attention_decoder timestep 8 of 100
INFO:tensorflow:Adding attention_decoder timestep 9 of 100
INFO:tenso

INFO:tensorflow:Restoring parameters from /home/ubuntu/W266/final_0/W266_Final/model_4/saved/train/model-40427
    [batch 35]: seen 3500 examples : 41.5 eps, Loss: 3.312, Avg loss: 3.325, Best loss: 3.311, cov loss: 0.152
    [batch 42]: seen 4200 examples : 44.4 eps, Loss: 3.338, Avg loss: 3.324, Best loss: 3.311, cov loss: 0.168
    [batch 49]: seen 4900 examples : 46.7 eps, Loss: 3.348, Avg loss: 3.325, Best loss: 3.311, cov loss: 0.176
    [batch 56]: seen 5600 examples : 48.6 eps, Loss: 3.407, Avg loss: 3.326, Best loss: 3.311, cov loss: 0.167
    [batch 63]: seen 6300 examples : 50.2 eps, Loss: 3.150, Avg loss: 3.323, Best loss: 3.311, cov loss: 0.162
    [batch 70]: seen 7000 examples : 51.6 eps, Loss: 3.242, Avg loss: 3.323, Best loss: 3.311, cov loss: 0.167
    [batch 77]: seen 7700 examples : 52.7 eps, Loss: 3.279, Avg loss: 3.323, Best loss: 3.311, cov loss: 0.167
    [batch 84]: seen 8400 examples : 53.8 eps, Loss: 3.242, Avg loss: 3.326, Best loss: 3.311, cov loss: 0.164
 

    [batch 539]: seen 53900 examples : 65.5 eps, Loss: 3.469, Avg loss: 3.350, Best loss: 3.311, cov loss: 0.173
    [batch 546]: seen 54600 examples : 65.5 eps, Loss: 3.429, Avg loss: 3.347, Best loss: 3.311, cov loss: 0.176
    [batch 553]: seen 55300 examples : 65.6 eps, Loss: 3.412, Avg loss: 3.344, Best loss: 3.311, cov loss: 0.162
    [batch 560]: seen 56000 examples : 65.6 eps, Loss: 3.215, Avg loss: 3.341, Best loss: 3.311, cov loss: 0.149
    [batch 567]: seen 56700 examples : 65.6 eps, Loss: 3.258, Avg loss: 3.336, Best loss: 3.311, cov loss: 0.175
    [batch 574]: seen 57400 examples : 65.6 eps, Loss: 3.512, Avg loss: 3.345, Best loss: 3.311, cov loss: 0.182
    [batch 581]: seen 58100 examples : 65.7 eps, Loss: 3.346, Avg loss: 3.347, Best loss: 3.311, cov loss: 0.157
    [batch 588]: seen 58800 examples : 65.7 eps, Loss: 3.235, Avg loss: 3.346, Best loss: 3.311, cov loss: 0.173
    [batch 595]: seen 59500 examples : 65.7 eps, Loss: 3.424, Avg loss: 3.348, Best loss: 3.311,

    [batch 1050]: seen 105000 examples : 66.9 eps, Loss: 3.405, Avg loss: 3.346, Best loss: 3.311, cov loss: 0.157
    [batch 1057]: seen 105700 examples : 66.9 eps, Loss: 3.250, Avg loss: 3.347, Best loss: 3.311, cov loss: 0.160
    [batch 1064]: seen 106400 examples : 66.9 eps, Loss: 3.519, Avg loss: 3.350, Best loss: 3.311, cov loss: 0.164
    [batch 1071]: seen 107100 examples : 66.9 eps, Loss: 3.443, Avg loss: 3.348, Best loss: 3.311, cov loss: 0.176
    [batch 1078]: seen 107800 examples : 66.9 eps, Loss: 3.429, Avg loss: 3.345, Best loss: 3.311, cov loss: 0.177
    [batch 1085]: seen 108500 examples : 66.9 eps, Loss: 3.440, Avg loss: 3.347, Best loss: 3.311, cov loss: 0.161
    [batch 1092]: seen 109200 examples : 66.9 eps, Loss: 3.428, Avg loss: 3.343, Best loss: 3.311, cov loss: 0.176
    [batch 1099]: seen 109900 examples : 66.9 eps, Loss: 3.362, Avg loss: 3.346, Best loss: 3.311, cov loss: 0.160
    [batch 1106]: seen 110600 examples : 66.9 eps, Loss: 3.400, Avg loss: 3.344,

    [batch 1554]: seen 155400 examples : 67.4 eps, Loss: 3.191, Avg loss: 3.345, Best loss: 3.311, cov loss: 0.150
    [batch 1561]: seen 156100 examples : 67.4 eps, Loss: 3.447, Avg loss: 3.345, Best loss: 3.311, cov loss: 0.176
    [batch 1568]: seen 156800 examples : 67.4 eps, Loss: 3.345, Avg loss: 3.343, Best loss: 3.311, cov loss: 0.173
    [batch 1575]: seen 157500 examples : 67.4 eps, Loss: 3.512, Avg loss: 3.345, Best loss: 3.311, cov loss: 0.161
    [batch 1582]: seen 158200 examples : 67.4 eps, Loss: 3.376, Avg loss: 3.345, Best loss: 3.311, cov loss: 0.175
    [batch 1589]: seen 158900 examples : 67.4 eps, Loss: 3.415, Avg loss: 3.346, Best loss: 3.311, cov loss: 0.168
    [batch 1596]: seen 159600 examples : 67.4 eps, Loss: 3.246, Avg loss: 3.345, Best loss: 3.311, cov loss: 0.162
    [batch 1603]: seen 160300 examples : 67.4 eps, Loss: 3.200, Avg loss: 3.344, Best loss: 3.311, cov loss: 0.155
    [batch 1610]: seen 161000 examples : 67.4 eps, Loss: 3.342, Avg loss: 3.342,

...

    [batch 1264]: seen 126400 examples : 67.1 eps, Loss: 3.279, Avg loss: 3.311, Best loss: 3.293, cov loss: 0.151
    [batch 1271]: seen 127100 examples : 67.1 eps, Loss: 3.403, Avg loss: 3.313, Best loss: 3.293, cov loss: 0.157
    [batch 1278]: seen 127800 examples : 67.1 eps, Loss: 3.221, Avg loss: 3.312, Best loss: 3.293, cov loss: 0.158
    [batch 1285]: seen 128500 examples : 67.1 eps, Loss: 3.396, Avg loss: 3.312, Best loss: 3.293, cov loss: 0.161
    [batch 1292]: seen 129200 examples : 67.1 eps, Loss: 3.336, Avg loss: 3.315, Best loss: 3.293, cov loss: 0.159
    [batch 1299]: seen 129900 examples : 67.1 eps, Loss: 3.425, Avg loss: 3.315, Best loss: 3.293, cov loss: 0.168
    [batch 1306]: seen 130600 examples : 67.1 eps, Loss: 3.317, Avg loss: 3.316, Best loss: 3.293, cov loss: 0.171
    [batch 1313]: seen 131300 examples : 67.1 eps, Loss: 3.246, Avg loss: 3.314, Best loss: 3.293, cov loss: 0.162
    [batch 1320]: seen 132000 examples : 67.1 eps, Loss: 3.276, Avg loss: 3.313,

    [batch 1762]: seen 176200 examples : 67.2 eps, Loss: 3.391, Avg loss: 3.302, Best loss: 3.289, cov loss: 0.167
    [batch 1769]: seen 176900 examples : 67.2 eps, Loss: 3.110, Avg loss: 3.300, Best loss: 3.289, cov loss: 0.154
    [batch 1776]: seen 177600 examples : 67.2 eps, Loss: 3.386, Avg loss: 3.302, Best loss: 3.289, cov loss: 0.168
    [batch 1783]: seen 178300 examples : 67.2 eps, Loss: 3.418, Avg loss: 3.305, Best loss: 3.289, cov loss: 0.167
    [batch 1790]: seen 179000 examples : 67.2 eps, Loss: 3.218, Avg loss: 3.305, Best loss: 3.289, cov loss: 0.172
    [batch 1797]: seen 179700 examples : 67.2 eps, Loss: 3.345, Avg loss: 3.312, Best loss: 3.289, cov loss: 0.174
    [batch 1804]: seen 180400 examples : 67.2 eps, Loss: 3.261, Avg loss: 3.311, Best loss: 3.289, cov loss: 0.178
    [batch 1811]: seen 181100 examples : 67.2 eps, Loss: 3.301, Avg loss: 3.312, Best loss: 3.289, cov loss: 0.173
    [batch 1818]: seen 181800 examples : 67.2 eps, Loss: 3.272, Avg loss: 3.317,

    [batch 2266]: seen 226600 examples : 67.6 eps, Loss: 3.370, Avg loss: 3.307, Best loss: 3.289, cov loss: 0.155
    [batch 2273]: seen 227300 examples : 67.6 eps, Loss: 3.220, Avg loss: 3.308, Best loss: 3.289, cov loss: 0.149
    [batch 2280]: seen 228000 examples : 67.6 eps, Loss: 3.474, Avg loss: 3.310, Best loss: 3.289, cov loss: 0.170
    [batch 2287]: seen 228700 examples : 67.6 eps, Loss: 3.176, Avg loss: 3.306, Best loss: 3.289, cov loss: 0.151
    [batch 2294]: seen 229400 examples : 67.6 eps, Loss: 3.300, Avg loss: 3.304, Best loss: 3.289, cov loss: 0.168
    [batch 2301]: seen 230100 examples : 67.6 eps, Loss: 3.157, Avg loss: 3.302, Best loss: 3.289, cov loss: 0.148
    [batch 2308]: seen 230800 examples : 67.6 eps, Loss: 3.349, Avg loss: 3.305, Best loss: 3.289, cov loss: 0.163
    [batch 2315]: seen 231500 examples : 67.6 eps, Loss: 3.480, Avg loss: 3.309, Best loss: 3.289, cov loss: 0.164
    [batch 2322]: seen 232200 examples : 67.6 eps, Loss: 3.007, Avg loss: 3.300,

    [batch 2766]: seen 276600 examples : 67.6 eps, Loss: 3.333, Avg loss: 3.303, Best loss: 3.286, cov loss: 0.152
    [batch 2773]: seen 277300 examples : 67.6 eps, Loss: 3.445, Avg loss: 3.304, Best loss: 3.286, cov loss: 0.167
    [batch 2780]: seen 278000 examples : 67.6 eps, Loss: 3.396, Avg loss: 3.306, Best loss: 3.286, cov loss: 0.155
    [batch 2787]: seen 278700 examples : 67.6 eps, Loss: 3.456, Avg loss: 3.308, Best loss: 3.286, cov loss: 0.160
    [batch 2794]: seen 279400 examples : 67.7 eps, Loss: 3.535, Avg loss: 3.314, Best loss: 3.286, cov loss: 0.176
    [batch 2801]: seen 280100 examples : 67.7 eps, Loss: 3.187, Avg loss: 3.310, Best loss: 3.286, cov loss: 0.153
    [END] Training complete: Total examples : 280600; Total time: 1:09:07
[EPOCH 25] Complete. Avg Loss: 3.31365515932395; Best Loss: 3.285648217685122
[EPOCH 26] Starting training..
    [batch 7]: seen 700 examples : 68.9 eps, Loss: 3.322, Avg loss: 3.313, Best loss: 3.286, cov loss: 0.161
    [batch 14]: se

KeyboardInterrupt: 

# Train session 8

In [6]:
avg_loss = 0
best_loss = None
curr_best = best_loss
train_step = 46184
epochs = 8
restore = True
epoch_start = 23

train_continue(hps,epochs,train_step,curr_best,best_loss,avg_loss,restore,epoch_start)







max_size of vocab was specified as 50000; we now have 50000 words. Stopping reading.
Finished constructing vocabulary of 50000 total words. Last word added: 16:03
INFO:tensorflow:Fetching data..
INFO:tensorflow:Creating batches..
INFO:tensorflow:[TOTAL Batches]  : 2808
INFO:tensorflow:[TOTAL Examples] : 280778
INFO:tensorflow:Creating batches..COMPLETE
INFO:tensorflow:Building core graph...
INFO:tensorflow:Adding attention_decoder timestep 0 of 100
INFO:tensorflow:Adding attention_decoder timestep 1 of 100
INFO:tensorflow:Adding attention_decoder timestep 2 of 100
INFO:tensorflow:Adding attention_decoder timestep 3 of 100
INFO:tensorflow:Adding attention_decoder timestep 4 of 100
INFO:tensorflow:Adding attention_decoder timestep 5 of 100
INFO:tensorflow:Adding attention_decoder timestep 6 of 100
INFO:tensorflow:Adding attention_decoder timestep 7 of 100
INFO:tensorflow:Adding attention_decoder timestep 8 of 100
INFO:tensorflow:Adding attention_decoder timestep 9 of 100
INFO:tenso

    [batch 35]: seen 3500 examples : 32.9 eps, Loss: 3.311, Avg loss: 3.214, Best loss: 3.187, cov loss: 0.151
    [batch 40]: seen 4000 examples : 34.1 eps, Loss: 3.370, Avg loss: 3.217, Best loss: 3.187, cov loss: 0.171
    [batch 45]: seen 4500 examples : 35.0 eps, Loss: 3.281, Avg loss: 3.217, Best loss: 3.187, cov loss: 0.160
    [batch 50]: seen 5000 examples : 35.9 eps, Loss: 3.265, Avg loss: 3.220, Best loss: 3.187, cov loss: 0.166
    [batch 55]: seen 5500 examples : 36.6 eps, Loss: 3.195, Avg loss: 3.223, Best loss: 3.187, cov loss: 0.169
    [batch 60]: seen 6000 examples : 37.2 eps, Loss: 3.373, Avg loss: 3.225, Best loss: 3.187, cov loss: 0.158
    [batch 65]: seen 6500 examples : 37.7 eps, Loss: 3.329, Avg loss: 3.231, Best loss: 3.187, cov loss: 0.148
    [batch 70]: seen 7000 examples : 38.2 eps, Loss: 3.345, Avg loss: 3.234, Best loss: 3.187, cov loss: 0.159
    [batch 75]: seen 7500 examples : 38.6 eps, Loss: 3.274, Avg loss: 3.235, Best loss: 3.187, cov loss: 0.150
 

    [batch 395]: seen 39500 examples : 43.8 eps, Loss: 3.500, Avg loss: 3.296, Best loss: 3.187, cov loss: 0.171
    [batch 400]: seen 40000 examples : 43.8 eps, Loss: 3.339, Avg loss: 3.297, Best loss: 3.187, cov loss: 0.149
    [batch 405]: seen 40500 examples : 43.8 eps, Loss: 3.355, Avg loss: 3.300, Best loss: 3.187, cov loss: 0.157
    [batch 410]: seen 41000 examples : 43.9 eps, Loss: 3.244, Avg loss: 3.298, Best loss: 3.187, cov loss: 0.162
    [batch 415]: seen 41500 examples : 43.9 eps, Loss: 3.291, Avg loss: 3.296, Best loss: 3.187, cov loss: 0.172
    [batch 420]: seen 42000 examples : 43.9 eps, Loss: 3.375, Avg loss: 3.295, Best loss: 3.187, cov loss: 0.163
    [batch 425]: seen 42500 examples : 43.9 eps, Loss: 3.384, Avg loss: 3.297, Best loss: 3.187, cov loss: 0.177
    [batch 430]: seen 43000 examples : 44.0 eps, Loss: 3.321, Avg loss: 3.297, Best loss: 3.187, cov loss: 0.171
    [batch 435]: seen 43500 examples : 44.0 eps, Loss: 3.034, Avg loss: 3.294, Best loss: 3.187,

    [batch 756]: seen 75600 examples : 44.7 eps, Loss: 3.437, Avg loss: 3.284, Best loss: 3.187, cov loss: 0.149
    [batch 761]: seen 76100 examples : 44.7 eps, Loss: 3.201, Avg loss: 3.284, Best loss: 3.187, cov loss: 0.167
    [batch 766]: seen 76600 examples : 44.7 eps, Loss: 3.099, Avg loss: 3.281, Best loss: 3.187, cov loss: 0.152
    [batch 771]: seen 77100 examples : 44.7 eps, Loss: 3.231, Avg loss: 3.282, Best loss: 3.187, cov loss: 0.171
    [batch 776]: seen 77600 examples : 44.7 eps, Loss: 3.212, Avg loss: 3.282, Best loss: 3.187, cov loss: 0.161
    [batch 781]: seen 78100 examples : 44.7 eps, Loss: 3.265, Avg loss: 3.282, Best loss: 3.187, cov loss: 0.156
    [batch 786]: seen 78600 examples : 44.7 eps, Loss: 3.397, Avg loss: 3.283, Best loss: 3.187, cov loss: 0.158
    [batch 791]: seen 79100 examples : 44.7 eps, Loss: 3.449, Avg loss: 3.283, Best loss: 3.187, cov loss: 0.171
    [batch 796]: seen 79600 examples : 44.8 eps, Loss: 3.359, Avg loss: 3.287, Best loss: 3.187,

    [batch 1117]: seen 111700 examples : 45.1 eps, Loss: 3.298, Avg loss: 3.293, Best loss: 3.187, cov loss: 0.167
    [batch 1122]: seen 112200 examples : 45.1 eps, Loss: 3.219, Avg loss: 3.296, Best loss: 3.187, cov loss: 0.146
    [batch 1127]: seen 112700 examples : 45.1 eps, Loss: 3.163, Avg loss: 3.296, Best loss: 3.187, cov loss: 0.156
    [batch 1132]: seen 113200 examples : 45.1 eps, Loss: 3.513, Avg loss: 3.292, Best loss: 3.187, cov loss: 0.171
    [batch 1137]: seen 113700 examples : 45.1 eps, Loss: 3.357, Avg loss: 3.292, Best loss: 3.187, cov loss: 0.168
    [batch 1142]: seen 114200 examples : 45.1 eps, Loss: 3.232, Avg loss: 3.292, Best loss: 3.187, cov loss: 0.156
    [batch 1147]: seen 114700 examples : 45.1 eps, Loss: 3.284, Avg loss: 3.291, Best loss: 3.187, cov loss: 0.172
    [batch 1152]: seen 115200 examples : 45.1 eps, Loss: 3.383, Avg loss: 3.291, Best loss: 3.187, cov loss: 0.167
    [batch 1156]: seen 115600 examples : 45.1 eps, Loss: 3.527, Avg loss: 3.291,

...

    [batch 1693]: seen 169300 examples : 46.4 eps, Loss: 3.319, Avg loss: 3.183, Best loss: 3.159, cov loss: 0.155
    [batch 1698]: seen 169800 examples : 46.4 eps, Loss: 3.265, Avg loss: 3.181, Best loss: 3.159, cov loss: 0.140
    [batch 1703]: seen 170300 examples : 46.4 eps, Loss: 3.428, Avg loss: 3.183, Best loss: 3.159, cov loss: 0.180
    [batch 1708]: seen 170800 examples : 46.4 eps, Loss: 3.138, Avg loss: 3.184, Best loss: 3.159, cov loss: 0.159
    [batch 1713]: seen 171300 examples : 46.4 eps, Loss: 3.068, Avg loss: 3.186, Best loss: 3.159, cov loss: 0.166
    [batch 1718]: seen 171800 examples : 46.4 eps, Loss: 3.124, Avg loss: 3.185, Best loss: 3.159, cov loss: 0.149
    [batch 1723]: seen 172300 examples : 46.4 eps, Loss: 3.028, Avg loss: 3.186, Best loss: 3.159, cov loss: 0.163
    [batch 1728]: seen 172800 examples : 46.4 eps, Loss: 3.212, Avg loss: 3.190, Best loss: 3.159, cov loss: 0.159
    [batch 1733]: seen 173300 examples : 46.4 eps, Loss: 3.254, Avg loss: 3.189,

    [batch 2050]: seen 205000 examples : 46.4 eps, Loss: 3.001, Avg loss: 3.182, Best loss: 3.159, cov loss: 0.145
    [batch 2055]: seen 205500 examples : 46.4 eps, Loss: 3.283, Avg loss: 3.186, Best loss: 3.159, cov loss: 0.163
    [batch 2060]: seen 206000 examples : 46.4 eps, Loss: 3.133, Avg loss: 3.186, Best loss: 3.159, cov loss: 0.158
    [batch 2065]: seen 206500 examples : 46.4 eps, Loss: 3.219, Avg loss: 3.190, Best loss: 3.159, cov loss: 0.161
    [batch 2070]: seen 207000 examples : 46.4 eps, Loss: 3.306, Avg loss: 3.191, Best loss: 3.159, cov loss: 0.165
    [batch 2075]: seen 207500 examples : 46.4 eps, Loss: 3.163, Avg loss: 3.191, Best loss: 3.159, cov loss: 0.148
    [batch 2080]: seen 208000 examples : 46.4 eps, Loss: 3.054, Avg loss: 3.191, Best loss: 3.159, cov loss: 0.157
    [batch 2085]: seen 208500 examples : 46.4 eps, Loss: 3.296, Avg loss: 3.194, Best loss: 3.159, cov loss: 0.183
    [batch 2090]: seen 209000 examples : 46.4 eps, Loss: 3.198, Avg loss: 3.194,

    [batch 2406]: seen 240600 examples : 46.4 eps, Loss: 3.260, Avg loss: 3.174, Best loss: 3.159, cov loss: 0.163
    [batch 2411]: seen 241100 examples : 46.4 eps, Loss: 3.069, Avg loss: 3.176, Best loss: 3.159, cov loss: 0.145
    [batch 2416]: seen 241600 examples : 46.4 eps, Loss: 3.185, Avg loss: 3.179, Best loss: 3.159, cov loss: 0.168
    [batch 2421]: seen 242100 examples : 46.4 eps, Loss: 3.283, Avg loss: 3.185, Best loss: 3.159, cov loss: 0.146
    [batch 2426]: seen 242600 examples : 46.4 eps, Loss: 3.104, Avg loss: 3.185, Best loss: 3.159, cov loss: 0.167
    [batch 2431]: seen 243100 examples : 46.4 eps, Loss: 3.123, Avg loss: 3.188, Best loss: 3.159, cov loss: 0.166
    [batch 2436]: seen 243600 examples : 46.4 eps, Loss: 3.073, Avg loss: 3.187, Best loss: 3.159, cov loss: 0.132
    [batch 2441]: seen 244100 examples : 46.4 eps, Loss: 3.218, Avg loss: 3.183, Best loss: 3.159, cov loss: 0.156
    [batch 2446]: seen 244600 examples : 46.4 eps, Loss: 3.184, Avg loss: 3.182,

    [batch 2765]: seen 276500 examples : 46.4 eps, Loss: 3.265, Avg loss: 3.186, Best loss: 3.159, cov loss: 0.152
    [batch 2770]: seen 277000 examples : 46.4 eps, Loss: 3.237, Avg loss: 3.186, Best loss: 3.159, cov loss: 0.155
    [batch 2775]: seen 277500 examples : 46.4 eps, Loss: 3.214, Avg loss: 3.185, Best loss: 3.159, cov loss: 0.160
    [batch 2780]: seen 278000 examples : 46.4 eps, Loss: 3.227, Avg loss: 3.184, Best loss: 3.159, cov loss: 0.138
    [batch 2785]: seen 278500 examples : 46.4 eps, Loss: 3.299, Avg loss: 3.186, Best loss: 3.159, cov loss: 0.157
    [batch 2790]: seen 279000 examples : 46.4 eps, Loss: 3.247, Avg loss: 3.187, Best loss: 3.159, cov loss: 0.156
    [batch 2795]: seen 279500 examples : 46.4 eps, Loss: 3.070, Avg loss: 3.188, Best loss: 3.159, cov loss: 0.151
    [batch 2800]: seen 280000 examples : 46.4 eps, Loss: 3.142, Avg loss: 3.187, Best loss: 3.159, cov loss: 0.146
    [batch 2805]: seen 280500 examples : 46.4 eps, Loss: 2.997, Avg loss: 3.183,

KeyboardInterrupt: 